In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import pandas as pd
import numpy as np
from prefect import task, Flow
from datetime import timedelta
from prefect.schedules import IntervalSchedule

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from prefect import task, Flow, Parameter, Client
from prefect.run_configs import KubernetesRun
from prefect.schedules import IntervalSchedule
from prefect.storage import S3
from prefect.tasks.control_flow import merge
from prefect import task, Flow, case
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from datetime import timedelta
import numpy as np
import pandas as pd
import mlflow
import requests
from typing import Tuple
import logging
import h5py
import sqlite3
import time
#import libraries
import streamlit as st
import pandas as pd
import numpy as np
import requests
from sklearn.ensemble import RandomForestClassifier
import json
import datetime
import streamlit as st
from pymongo import MongoClient, errors
from PIL import Image 
import requests
from io import BytesIO

In [8]:

def calculate_rmse():
    
    filename = "best_predictions.h5"

    with h5py.File(filename, "r") as f:
        # Print all root level object names (aka keys) 
        # these can be group or dataset names 
        print("Keys: %s" % f.keys())
        # get first object name/key; may or may NOT be a group
        a_group_key = list(f.keys())[0]

        # get the object type for a_group_key: usually group or dataset
        print(type(f[a_group_key])) 

        # If a_group_key is a group name, 
        # this gets the object names in the group and returns as a list
        data = list(f[a_group_key])

        # If a_group_key is a dataset name, 
        # this gets the dataset values and returns as a list
        data = list(f[a_group_key])
        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key][()]  # returns as a numpy array
        
        # Use True lable to calculate RMSE
        return 0.7
    


In [9]:
@task(max_retries=3, retry_delay=timedelta(seconds=10))
def is_model_degraded_condition():
    rmse = calculate_rmse()
    if rmse > 0.4:
        return True
    else:
        return False  

    
@task(max_retries=3, retry_delay=timedelta(seconds=10))
def inference():
    os.system("sbatch ./../../inference_harsh_cp_1.sh")    
    time.sleep(30)
    
@task(max_retries=3, retry_delay=timedelta(seconds=10))
def save_to_database():
    pass

@task(max_retries=3, retry_delay=timedelta(seconds=10))
@st.experimental_singleton
def init_connection(): 
        client = MongoClient(host='localhost:27017' , serverselectiontimeoutms=3000)
        return client
    
@task(max_retries=3, retry_delay=timedelta(seconds=10))
def write_data(client):
        data_1 = {
        "rmse": 3.1, 
        "event_date":  datetime.datetime(2022, 8, 10),
        "image_url": "https://www.dkrz.de/en/about-en/aufgaben/dkrz-and-climate-research/@@images/image/large"
        }

        data_2 = {
        "rmse": 2.1,         
        "event_date":  datetime.datetime(2022, 8, 9),
        "image_url": "https://www.dkrz.de/en/about-en/aufgaben/dkrz-and-climate-research/@@images/image/large"
        }

        data_3 = {
        "rmse": 3.2,         
        "event_date":  datetime.datetime(2022, 8, 8),
        "image_url": "https://www.dkrz.de/en/about-en/aufgaben/dkrz-and-climate-research/@@images/image/large"
        }


        cygnss_collection = client["cygnss"].cygnss_collection

        cygnss_collection = cygnss_collection.insert_many([data_1, data_2, data_3])

        print(f"Multiple tutorials: {cygnss_collection.inserted_ids}")

@task
def get_data(client):        
        cygnss = client.cygnss                        
        items = cygnss.cygnss_collection.find()        
        items = list(items)  # make hashable for st.experimental_memo
        for item in items:
            print(f"RMSE is: {item['rmse']}")            
        
        
@task
def drop_database(client):
    client.drop_database('cygnss')

In [11]:
schedule = IntervalSchedule(interval=timedelta(minutes=2))


# 1. Get fresh data
# 2. Pre-process the data
# 3. Do a inference with that
# 4. If model is working above a set threshold. Fine. 
# 5. If not, then train the model on new data.
# 6. Evaluate the model and check if it is still fine or not and take specific action

# TODO:
# in case of failure of any task, retry 
# save the results in database

with Flow("MLOps") as flow:
    
    # download_data()
    # pre_process()
    
    
    client = init_connection()    
    drop_database(client)
    
    inference()
    
    # read the results and tell if the model is degraded
    is_model_degraded = is_model_degraded_condition()

    # train on new data if error is more than threshold and log the results on test data
    with case(is_model_degraded, True): 
#         model = train_model_task(train_data)        
        inference()
        is_model_degraded_condition()

    # save_to_db
    write_data(client)
    
#     get_data(client)
    
    
    
state = flow.run()
print('DONE')
# flow.visualize()

[2022-08-31 11:59:12+0200] INFO - prefect.FlowRunner | Beginning Flow run for 'MLOps'


2022-08-31 11:59:12.523 Beginning Flow run for 'MLOps'


[2022-08-31 11:59:12+0200] INFO - prefect.TaskRunner | Task 'init_connection': Starting task run...


2022-08-31 11:59:12.543 Task 'init_connection': Starting task run...


[2022-08-31 11:59:12+0200] INFO - prefect.TaskRunner | Task 'init_connection': Finished task run for task with final state: 'Success'


...

2022-08-31 11:59:42.720 Task 'write_data': Starting task run...


Multiple tutorials: [ObjectId('630f310e0e226a62e1eb5a5b'), ObjectId('630f310e0e226a62e1eb5a5c'), ObjectId('630f310e0e226a62e1eb5a5d')]
[2022-08-31 11:59:42+0200] INFO - prefect.TaskRunner | Task 'write_data': Finished task run for task with final state: 'Success'


2022-08-31 11:59:42.738 Task 'write_data': Finished task run for task with final state: 'Success'


[2022-08-31 11:59:42+0200] INFO - prefect.TaskRunner | Task 'inference': Starting task run...


2022-08-31 11:59:42.745 Task 'inference': Starting task run...
sh: 1: sbatch: not found

KeyboardInterrupt

